<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course5/week1/Improvise_a_Jazz_Solo_with_an_LSTM_Network_v3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Improvise a Jazz Solo with an LSTM Network

이번 주차 최종 과제에 오신것을 환영합니다! 이번 과제에서 여러분은 LSTM 신경망을 사용한 음악 생성 모델을 만들 것입니다. 이번 과제가 마무리되면, 여러분은 직접 제작한 음악을 들을 수 있을 것입니다.

**이번 과제에서 여러분은 다음의 내용을 배웁니다:**
- LSTM을 사용한 음악 생성 모델 구현하기
- 딥 러닝을 통해서 직접 재즈 뮤직 생성해보기

아래 코드 블록을 실행시켜 이번 과제에 필요한 패키지를 불러옵니다. 몇 분정도 걸릴 수 있습니다.

In [ ]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

## 1 - Problem statement

여러분은 친구의 생일을 축하기 위해 특별이 재즈 음악을 만들기로 해습니다. 하지만, 여러분은 작곡이나 악기 연주에 대해서 특별히 알고 있는 지식이 없죠. 다행히도, 여러분은 딥 러닝을 배웠고 이 문제를 해결하기 위해 LSTM 신경망으로 사용하려고 합니다.

여러분은 참신한 재즈 솔로를 생성하는 인공 신경망을 훈련하고자 합니다.

<img src="arts/jazz.jpg" style="width:450;height:300px;">



### 1.1 - Dataset

여러분은 재즈 음악 묶음을 사용해서 알고리즘을 훈련시키려고 합니다. 아래 코드 블록을 실행시켜 훈련 데이터 일부의 오디오 클립을 재생해 보세요.

In [ ]:
IPython.display.Audio('./data/30s_seq.mp3')

우리는 음악 데이터의 "value" 라는 관점에서 데이터를 랜더링하기 위해, 음악 데이터의 전처리를 수행했습니다.

#### Details about music (optional)
여기서, 각 "value"는 음높이와 지속 시간으로 구성된 음표로 생각할 수 있습니다. 예를 들어 특정 피아노 건반을 0.5초동안 누르면, 그 음표를 연주한 것입니다. 음악 이론에서 "value"라는 개졈은 실제로는 이보다 더 복잡합니다. 특히 동시에 여러 음을 연주하는 필요한 정보도 필요합니다. 예를 들어, 악보를 연주할 때 동시에 두 개 이상의 피아노 건반을 누를 수 있습니다(동시에 여러 음을 연주하면, "코드"라고 하는 것이 생성됩니다). 그러나 우리는 이 과제를 위해 음악 이론의 세부 사항에 대해 깊게 알고 있을 필요는 없습니다.

#### Music as a sequence of values
* 이 과제를 수행하기 위해 알아야 할 것은 값의 데이터 셋을 얻고 값의 시퀀스를 생성하기 위해 RNN 모델을 학습한다는 것입니다.
* 우리의 음악 생성 모델은 78 개의 고유 한 value를 사용합니다.


다음 코드를 실행하여 원시 음악 데이터를 불러오고, "value" 라는 관점으로 전처리된 데이터로 변환합니다. 몇 분 정도 걸릴 수 있습니다.

In [ ]:
X, Y, n_values, indices_values = load_music_utils()
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('shape of X:', X.shape)
print('Shape of Y:', Y.shape)

위에서 불러온 데이터에 대해 간락히 설명하자면, 다음과 같습니다.

- **X**: 이 값은 $(m, T_x 78)$의 shape를 가진 배열입니다.
  - 우리는 총 $m$개의 훈련 데이터를 가지고 있습니다. 따라서 각각의 음악 스니펫(단편)은 $T_x = 30$ 개의 value를 가지고 있습니다.
  - 각 time step마다, 78개의 사용 가능한 모든 value값을 입력으로 받습니다. 이 값은 one-hot 벡터로 표현되어 있습니다.
    - 예를 들어, `X[i, t, :]` 는 i 번째 훈련 데이터의 t 번째 time step의 value를 나타냅니다.
- **Y**: $(T_y, m, 78)$ shape의 배열입니다.
  - 이 값은 본질적으로 X와 같지만, 왼쪽 (과거)으로 한 단계 이동했습니다.
  - Y의 데이터는 $(T_y, n, 78)$로, 배열의 shape가 재조정되었음을 기억하세요. 여기서 $T_y = T_x$ 입니다. 이 형태는 추후 LSTM에 데이터를 집어넣을 때 더 편리한 형태입니다.
  - 지난 공룡 이름짓기 과제와 비슷하게, 이전 값을 가지고 다음 값을 예측하는데 사용하겠습니다.
    - 따라서 우리의 순환 신경망은 주어진 $x^{\langle 1\rangle}, \ldots, x^{\langle t \rangle}$에 대하여 $y^{\langle t \rangle}$를 예측합니다.
- `n_values` : 서로 다른 musical value(위에서 설명됨)들의 갯수입니다. 이 값은 78입니다.
- `indices_values` : 0부터 77까지, musical value를 각각의 정수로 맵핑하는 파이썬 딕셔너리입니다.

### 1.2 - Overview of our model

아래 그림은 우리가 사용할 모델의 구조입니다. 이는 이번 과제에선 케라스를 사용한다는 점을 제외하면, 지난 공룡 이름 짓기 과제와 유시합니다.

<img src="arts/music_generation.png" style="width:600;height:400px;">

- $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, \cdots, x^{\langle T_x \rangle})$는 음악 묶음을 스캔해서 구할 수 있는 $T_x$ 크기의 창입니다.
- 각 $x^{\langle t \rangle}$ 는 특정 time step t에 해당하는 musical value의 인덱스입니다.
- $\hat y^t$는 다음 value에 대한 예측 결과입니다.
- 우리는 훨씬 긴 음악에서 가져온 30 개 값의 무작위 스 니펫으로 모델을 훈련 할 것입니다.
  - 그러므로, 대부분의 음악 스니펫은 음악 중간에서부터 시작하므로  첫 번째 입력 $x^{\langle 1 \rangle} = \vec{0}$를 설정하지 않아도 됩니다.
  - 벡터화를 더 쉽게 하기 위해 각 조각의 길이를 $T_x = 30$으로 동일하게 설정합니다.


## Overview of parts 2 and 3

* 우리는 훈련 된 재즈 음악과 유사한 스타일로 다음 음을 예측하는 모델을 훈련 할 것입니다. 훈련은 모델의 가중치와 bias를 포함됩니다.
* Part 3에서는 일련의 음표를 예측하는 새로운 모델에서 가중치와 bias를 사용하고 이전 음표를 사용하여 다음 음표를 예측할 것입니다.
* 가중치와 bias는 아래 설명 된 '전역 공유 레이어(global shared layers)'를 사용하여 새 모델로 이전됩니다.

## 2 - Building the model

- 이 파트에서 여러분은 음악적 패턴을 학습하는 모델을 구현하고 훈련시킬 것입니다.
- 모델은 $(m, T_x, 78)$의 shape를 가지는 배열 X를 입력으로 받아, $(T_y, m, 78)$의 shape를 가지는 배열 Y를 출력합니다.
- 우리는 $n_{a} = 64$ shape의 LSTM hidden state를 사용합니다.

In [ ]:
# number of dimensions for the hidden state of each LSTM cell.
n_a = 64 

#### Sequence generation uses a for-loop
- 만약 테스트 타임에 RNN 모델을 빌드하는 경우 전체 입력 시퀸스 $x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, \ldots, x^{\langle T_x \rangle}$를 사전에 입력하면, 케라스에서는 간단히 모델을 만들어주는 빌트인 함수를 제공합니다.
- 하지만, **시퀸스 생성 알고리즘에서, 테스트 타임에 $x^{\langle t\rangle}$ 값을 사전에 알 수 없는 경우가 있습니다**
- 대신 $x^{\langle t\rangle} = y^{\langle t-1 \rangle}$를 사용하여 한번에 한개 씩 생성합니다.
  - "t" time에서의 입력은 이전 time step인 "t-1"의 예측 결과입니다.
- 따라서 모든 time step을 반복하려면 for 루프를 구현해야합니다.


#### Shareable weights

- `djmodel()` 함수는 반복문을 사용하여 $T_x$ time 횟수 만큼 LSTM 레이어를 호출하는 함수입니다.
- 여기서 $T_x$의 복사본이 같은 가중치를 가진다는 것이 중요합니다.
  - $T_x$번의 반복 단계 동안, 가중치들은 매번 새롭게 초기화되는 것이 아니라 공유되는 값을 가져야 합니다.
- 전역적으로 선언된 shared layer를 참조하면 각 time step에서 동일한 레이어 인스턴스를 사용합니다.
- 케라스에서 공유 가능한 가중치를 사용하여 레이어를 구현하는 주요 단계는 다음과 같습니다.
  1. 레이어 객체를 정의합니다(이를 위해 전역 변수를 사용합니다).
  2. 입력값을 받아 propagation할 때 위의 객체를 호출합니다.

#### Shareable weights

- 필요한 레이어 객채를 전역 변수로 정의했습니다.
- 위의 객체를 생성하려면 아래 코드 블록을 실행시키세요.
- 케라스 문서를 읽고 다음의 레이어에 대해 알아보세요.
  - [Reshape()](https://keras.io/layers/core/#reshape) : 출력을 특정 모양으로 재구성합니다.
  - [LSTM()](https://keras.io/layers/recurrent/#lstm) : Long-Short Term Memory
  - [Dense()](https://keras.io/layers/core/#dense) : 완전히 연결된 신경망 계층

In [ ]:
n_values = 78 # number of music values
reshapor = Reshape((1, n_values))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

- `reshapor`, `LSTM_cell`과 `densor`는 전역적으로 선언된 레이어 객체입니다. 따라서 이 값을 `djmodel()` 함수를 구현하는데 사용할 수 있습니다.
- 케라스 텐서 객체 X를 위의 세 객체를 통해 propagate하기 위해서는 `layer_object()` 함수를 사용하세요.
  - 하나의 입력값에 대해서는 `layer_object(X)`를 사용합니다.
  - 하나 이상의 입력에 대해서는 리스트를 인자로 집어넣습니다, : `layer_object([X1, X2])`


**연습 문제:** `djmodel()` 함수를 구현하세요.

#### Inputs(given)

- `Input()` 레이어는 입력 X와 최초의 hidden state 'a0'와 cell state 'c0'를 정의하기 위해서 사용됩니다.
- `shape` 파라미터는 배치 사이즈에 대한 정보 (m)을 포함하고 있지 않는 튜플입니다.
  - 예를 들어,
  ```Python
    X = Input(shape=(Tx, n_values)) # X has 3 dimensions and not 2: (m, Tx, n_values)
  ```

#### Step 2: Outputs(TODO)
1. 빈 리스트인 "outputs"를 만들고, LSTM cell을 거쳐서 나온 결과들을 해당 리스트에 추가합니다.

#### Step 2: Loop through time steps(TODO)
- $t \in 1, \ldots, T_x$와 같이 반복하세요.

#### 2A. Select the 't' time-step vector from X

- X는 `(m, Tx, n_values)`의 shape를 하고 있습니다.
- 't' 선택의 모양은 (n_values,)이어야합니다.
- Keras 대신 numpy로 구현하는 경우 다음과 같이 3D numpy 배열에서 슬라이스를 추출합니다.
```Python
var1 = array1[:,1,:]
```

#### Lambda layer

* Keras를 사용하고 있으므로 커스텀 레이어 내에서 이 단계를 정의해야합니다.
* Keras에서 이것은 Lambda 계층 [Lambda](https://keras.io/layers/core/#lambda)입니다.
* 예를 들어 이전 레이어를 가져 와서 '1'을 추가하는 Lambda 레이어는 다음과 같습니다.
```python
lambda_layer1 = Lambda(lambda z: z + 1)(previous_layer)
``` 
* 이 경우 이전 레이어는`X`입니다.
* `z`는 람다 함수의 지역 변수입니다.
  * `previous_layer`는 소문자 `lambda`함수의 매개 변수 `z`로 전달됩니다.
  * 원하는 경우 변수 이름을 다른 이름으로 선택할 수 있습니다.
* 콜론 ':'뒤의 연산은 이전 레이어에서 슬라이스를 추출하는 연산이어야합니다.
* **힌트** : Lambda에 인수로 전달되지 않더라도 람다 계층의 정의 내에서 `t` 변수를 사용하게됩니다.

#### 2B. Reshape x to be (1,n_values).
- `reshapor()` 레이어를 사용하세요. 이는 지난 레이어의 출력값을 받아 입력 인자로 추가하는 역할을 합니다.

#### 2C. Run x through one step of LSTM_cell.
- 이전 단계의 hidden state $a$ 및 cell state $c$로 `LSTM_cell`을 초기화합니다.
- 다음 공식을 사용하세요.
```python
next_hidden_state, _, next_cell_state = LSTM_cell(inputs=input_x, initial_state=[previous_hidden_state, previous_cell_state])
```
- input, hidden state, cell state로 적절한 변수를 선택하세요.

#### 2D. Dense layer
- LSTM의 hidden state를 `densor` 레이어를 사용하여 dense + softmax 레이어를 통과시키는 propagate 연산을 수행하세요.

#### 2E. Append output
- 최종 출력 결과를 "outputs" 리스트에 추가하세요.

#### Step 3: After the loop, create the model

- 케라스의 `Model` 객체를 사용하여 모델을 만듭니다.
- 입력 및 출력 지정 :
```Python
model = Model(inputs=[input_x, initial_hidden_state, initial_cell_state], outputs=the_outputs)
```
- input, hidden state, cell state로 적절한 변수를 선택하세요.
- [Model](https://keras.io/models/model/) 문서에서 더 많은 정보를 확인하세요.


In [ ]:
# GRADED FUNCTION: djmodel

def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras instance model with n_a activations
    """
    
    # Define the input layer and specify the shape
    X = Input(shape=(Tx, n_values))
    
    # Define the initial hidden state a0 and initial cell state c0
    # using `Input`
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = None
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = None
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = None
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = None
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = None
        # Step 2.E: add the output to "outputs"
        None
        
    # Step 3: Create model instance
    model = None
    
    ### END CODE HERE ###
    
    return model

#### Create the model object

- 아래 코드 블록을 실행시켜 모델을 정의해보세요.
- `Tx=30`, `n_a=64`(LSTM 활성화 변수의 차원)으로 설정하고, `n_values=78`로 설정하겠습니다.
- 이 코드 블록은 실행하는데 수 초가 걸릴 수 있습니다.

In [ ]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

In [ ]:
# Check your model
model.summary()

**모범 답안**  
출력 결과를 맨 아래까지 스크롤하면, 아래 정보를 확인하실 수 있습니다.

```Python
Total params: 41,678
Trainable params: 41,678
Non-trainable params: 0
```

#### Compile the model for training

- 이제 학습시킬 모델을 컴파일해야 합니다.
- 아래 두 가지 개념을 사용할 것입니다.
  - 최적화 함수 : Adam 최적화 함수
  - 손실 함수 : 범주형(categorical) 크로스 엔트로피 함수(다중 클래스 분류)

In [ ]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

#### Initialize hidden state and cell state

최종적으로, LSTM의 초기 상태를 0으로 만들기 위해서 `a0`와 `c0`의 값을 초기화해봅시다.

In [ ]:
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

#### Train the model

- 이제 모델을 훈련시켜봅니다.
- 비용함수가 `Y`를 리스트의 형태로 받기 때문에, `Y`를 리스트로 변환합니다.
  - `list(Y)`라는 코드는 30개의 아이템을 가진 리스트입니다. 각각의 아이템은 shape가 (60, 78)인 데이터입니다.
  - 100회의 epoch동안 훈련을 해봅시다. 이 작업은 몇 분 정도 소요될 수 있습니다.

In [ ]:
model.fit([X, a0, c0], list(Y), epochs=100)

#### 모범 답안

이 모델의 손실은 높게 (100 개 정도) 시작했다가, epoch 100회가 지나면 한자릿수 여야 합니다. 가중치의 무작위 초기화로 인해 아래의 모범 답안은 정확한 숫자가 아닙니다.
예를 들면 :
```
Epoch 1/100
60/60 [==============================] - 3s - loss: 125.7673
...
```
Scroll to the bottom to check Epoch 100
```
...
Epoch 100/100
60/60 [==============================] - 0s - loss: 6.1861
```


모델을 학습 했으므로 추론 알고리즘을 구현하고 음악을 생성하는 마지막 섹션으로 이동하겠습니다!

## 3 - Generating music

지금까지 재즈 솔로 세션의 연주 패턴을 학습하는 모델을 훈련시켰습니다. 이제 이 모델을 사용해서 새로운 음악을 합성해보도록 하겠습니다.

#### 3.1 - Predicting & Sampling

<img src="arts/music_gen.png" style="width:600;height:400px;">

Sampling의 각 단계마다 여러분은,
- 이전 LSTM으로부터 활성화 변수 `a`와 cell state `c`를 입력으로 받습니다.
- 한 단계의 forward propagation 연산을 수행합니ㅏㄷ.
- 새로운 활성화 변수와 cell state를 얻어냅니다.
- 그 다음 새로운 활성화 변수 `a`는를 사용해 fully connected 레이어인 `densor`를 사용해 출력을 생성할 수 있습니다.


##### Initialization
- 아래의 변수들을 0으로 초기화합니다.
  - `x0`
  - hidden state `a0`
  - cell state `c0`



**연습 문제**
- musical value의 시퀸스를 샘플링하기 위한 아래 함수를 구현하세요.
- 다음은 $T_y$ 출력 문자열을 생성하는 반복문 내부에서 구현해야 하는 몇가지 주요 단계입니다.
- 단계 2.A : 입력 레이어를 받는 `LSTM_cell`과 이전 단계의 `c`, `a`를 사용하여 현재 단계의 `c`와 `a`를 생성합니다.
```python
next_hidden_state, _, next_cell_state = LSTM_cell (input_x, initial_state = [previous_hidden_state, previous_cell_state])
```
  - `input_x`, `hidden_state` 및 `cell_state`에 대해 적절한 값을 선택하세요.
- 단계 2.B : 현재 단계에 대한 출력을 얻기 위해, `a`에 대한 softmax 함수를 계산하기 위해 `densor` 객체를 적용하여 출력을 계산합니다.
- 단계 2.C : 계산된 출력을 "outputs" 리스트에 추가합니다.


- 단계 2.D : `x`를 출력 결과인 `out`의 one-hot 인코딩 버전의 값으로 샘플링합니다.
- 위 작업은 다음 LSTM 계산에 도움이 됩니다.
- 이번 과제에서는, `music_utils.py` 파일 안에 `one_hot(x)`라는 함수를 미리 정의해놓았습니다. `one_hot` 함수의 코드는 다음과 같습니다.
```python
def one_hot(x):
  x = K.argmax(x)
  x = tf.one_hot(indices=x, depth=78) 
  x = RepeatVector(1)(x)
  return x
```
다음은 `one_hot` 함수가 어떻게 동작하는지에 대한 내용입니다.
- `argmax` : 주어진 벡터 `x`에 대하여, 최댓값과 해당 최댓값이 위치한 벡터의 인덱스 값을 리턴합니다.
  - 예를 들어, `[-1,0,1]` 배열에 대한 `argmax` 함수는 최댓값 1과 그 인덱스인 2를 리턴합니다. 자세한 사항은 [keras.argmax]((https://www.tensorflow.org/api_docs/python/tf/keras/backend/argmax)) 문서를 참조하세요.
- `one_hot` : 인덱스 목록과 one-hot 벡터의 깊이(카테고리의 수 : 이번 과제에서는 78개)를 인자로 받습니다. 각 인덱스를 one-hot 인코딩으로 변환합니다.
  - 예를 들어, 인덱스가 2이고 깊이가 5이면 one-hot 벡터는 `[0,0,1,0,0]` 입니다. 더 많은 예제와 설명은 [tf.one_hot](https://www.tensorflow.org/api_docs/python/tf/one_hot)에 대한 문서를 확인하세요.
- `RepeatVector(n)` : 벡터를 취하여 n번 복제합니다. 이번에는 1번 반복했습니다. 이 작업은 아무것도 하지 않는 것처럼 보일 수 있습니다. [RepeatVector](https://keras.io/layers/core/#repeatvector)에 대한 문서를 보면 x가 (m, 5) 차원의 벡터고, `RepeatVector(1)`로 전달된다는 것을 알 수 있습니다. `RepeatVector(1)`이면 출력은 (m,1,5)입니다. 즉, 결과 벡터에 차원(길이 1)을 추가합니다.
- [Lambda](https://keras.io/layers/core/#lambda) 레이어를 사용하여 커스텀 one_hot 인코딩을 적용합니다. 앞에서 Lambda 계층을 다음과 같이 사용할 수 있음을 확인했습니다.
```python
result = Lambda(lambda x : x + 1)(input_var)
```

만약 여러분이 사전에 정의된 함수를 사용한다면, 다음과 같이 할 수 있습니다.
```python
def add_one(x)
    return x + 1

# use the add_one function inside of the Lambda function
result = Lambda(add_one)(input_var)
```

#### Step 3 : Inference Model

아래는 케라스의 `Model` 객체를 어떻게 사용하는지 다룬 코드입니다.
```Python
model = Model(inputs=[input_x, initial_hidden_state, initial_cell_state], outputs=the_outputs)
```

- 입력 텐서, hidden state, cell state와 출력에 대해서 적절한 변수를 대입하세요.
- **힌트** : 모델의 입력값은 **초기 상태의** input과 state입니다.

In [ ]:
# GRADED FUNCTION: music_inference_model

def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, number of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = None
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(None):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = None
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = None

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        None
        
        # Step 2.D: 
        # Select the next value according to "out",
        # Set "x" to be the one-hot representation of the selected value
        # See instructions above.
        x = None
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = None
    
    ### END CODE HERE ###
    
    return inference_model

추론 모델을 정의하려면 아래 셀을 실행하십시오. 이 모델은 50 개의 musical value를 생성하도록 하드 코딩되어 있습니다.

In [ ]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

In [ ]:
# Check the inference model
inference_model.summary()

**모범 답안**  
출력 결과를 맨 아래까지 스크롤하면, 아래 정보를 확인하실 수 있습니다.

```Python
Total params: 41,678
Trainable params: 41,678
Non-trainable params: 0
```

#### Initialize inference model

다음 코드는 x와 LSTM 상태 변수 a 및 c를 초기화하는 데 사용할 값이 0 인 벡터를 만듭니다.

In [ ]:
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

**연습 문제** : `predict_and_sample()` 함수를 구현합니다.

* 이 함수는 입력 [x_initializer, a_initializer, c_initializer]를 포함하여 많은 인자를 취합니다.
* 이 입력에 해당하는 출력을 예측하려면 다음 3 단계를 수행해야합니다.

#### Step 1
- 모델과 입력 데이터 세트를 사용하여 출력을 예측하십시오. 출력 `pred`는 길이 $ T_y $의 리스트입니다. 여기서 각 리스트의 각 요소는 (1, n_values) shape의 numpy 배열 입니다.
```Python
inference_model.predict([input_x_init, hidden_state_init, cell_state_init])
```
  - 이 `predict_and_sample` 함수의 입력 인자로 집어넣을 `prdeict` 변수에할 적절한 값을 선택합니다.

#### Step 2


- `pred`를 $ T_y $ 인덱스의 numpy 배열로 변환합니다.
  - 각 인덱스는 `pred` 리스트의 각 요소의 `argmax`를 취하여 계산됩니다.
  - [numpy.argmax](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html)를 사용합니다.
  - `axis` 매개 변수를 설정합니다.
    - 예측 결과는 $ (m, T_ {y}, n_ {values}) $ shape의 배열입니다.

#### Step 3


* 인덱스를 원-핫 벡터 표현으로 변환합니다.
  * [to_categorical](https://keras.io/utils/#to_categorical)을 사용합니다.
  * `num_classes` 파라미터를 설정합니다. 채점을 위해 다음 중 하나가 필요합니다.
    * `predict_and_sample()` 함수의 지정된 매개 변수를 사용합니다. 예 : x_initializer의 측정 기준 중 하나에 고유 한 클래스 수 값이 있음).
    * 또는 개별 클래스의 수를 하드 코딩하십시오 (채점에서 도 통과 함).
    * `n_values` 와 같은 전역 변수를 사용하면 채점 알고리즘에서 작동하지 않습니다.

In [ ]:
# GRADED FUNCTION: predict_and_sample

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = None
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = None
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (Ty, n_values)
    results = None
    ### END CODE HERE ###
    
    return results, indices

In [ ]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

**모범 답안 :**

* Keras의 결과는 완전히 예측할 수 없기 때문에 **결과가 다를 수 있습니다**.
* 그러나 위에서 설명한대로 정확히 100 epoch 동안 `model.fit()`을 사용하여 LSTM_cell을 훈련 한 경우 :
  * 모두 동일하지 않은 일련의 인덱스를 관찰해야합니다.
  * 또한 다음 사항을 준수해야합니다.
    * np.argmax (results [12])는 list (indices [12:18])의 첫 번째 요소입니다.
    * 및 np.argmax (results [17])는 list (indices [12:18])의 마지막 요소입니다.

<table>
    <tr>
        <td>
            **np.argmax(results[12])** =
        </td>
        <td>
        1
        </td>
    </tr>
    <tr>
        <td>
            **np.argmax(results[17])** =
        </td>
        <td>
        42
        </td>
    </tr>
    <tr>
        <td>
            **list(indices[12:18])** =
        </td>
        <td>
            [array([1]), array([42]), array([54]), array([17]), array([1]), array([42])]
        </td>
    </tr>
</table>

### 3.3 - Generate music

마지막으로 음악을 생성 할 준비가되었습니다. RNN은 일련의 값을 생성합니다. 다음 코드는 먼저 `predict_and_sample ()` 함수를 호출하여 음악을 생성합니다. 이 값은 음악 코드로 후 처리됩니다 (즉, 여러 값 또는 음표를 동시에 연주 할 수 있음).

대부분의 컴퓨터 음악 알고리즘은 이러한 후 처리없이 좋은 소리를내는 음악을 생성하기 어렵 기 때문에 약간의 후 처리를 사용합니다. 포스트 프로세싱은 동일한 사운드가 너무 많이 반복되지 않도록하여 생성 된 오디오를 정리하는 것과 같은 작업을 수행합니다. 두 개의 연속 된 음표가 피치에서 너무 멀지 않은지 확인합니다. 이러한 사후 처리 단계 중 상당수가 해킹이라고 주장 할 수 있습니다. 또한 많은 음악 생성 문헌은 수작업으로 포스트 프로세서를 만드는 데 중점을 두 었으며 많은 출력 품질은 RNN의 품질뿐만 아니라 포스트 프로세싱의 품질에 달려 있습니다. 그러나이 후 처리는 큰 차이를 만들므로 우리 구현에서도 사용합시다.

음악을 만들어 보겠습니다!

다음 셀을 실행하여 음악을 생성하고 `out_stream`에 녹음합니다. 몇 분 정도 걸릴 수 있습니다.

In [ ]:
out_stream = generate_music(inference_model)

음악을 들으려면 파일-> 열기 ...를 클릭 한 다음 "output /"으로 이동하여 "my_music.midi"를 다운로드합니다. 미디 파일이있는 경우이를 읽을 수있는 응용 프로그램으로 컴퓨터에서 재생하거나 무료 온라인 "MIDI to mp3"변환 도구 중 하나를 사용하여이를 mp3로 변환하십시오.

참고로이 알고리즘을 사용하여 생성 한 30 초 오디오 클립이 있습니다.

In [ ]:
IPython.display.Audio('./data/30s_trained_model.mp3')

### Congratulations!

이 과제를 마무리했습니다.


## What you should remember

- 시퀀스 모델을 사용하여 musical value를 생성 한 다음 MIDI 음악으로 후 처리 할 수 ​​있습니다.
- 비슷한 모델을 사용하여 공룡 이름을 생성하거나 음악을 생성 할 수 있으며, 주요 차이점은 모델에 투입되는 입력입니다.
- Keras에서 시퀀스 생성에는 공유 가중치를 사용하여 레이어를 정의하는 작업이 포함되며, 이는 time step $ 1, \ldots, T_x $만큼 반복됩니다.

**References**

The ideas presented in this notebook came primarily from three computational music papers cited below. The implementation here also took significant inspiration and used many components from Ji-Sung Kim's GitHub repository.

- Ji-Sung Kim, 2016, [deepjazz](https://github.com/jisungk/deepjazz)
- Jon Gillick, Kevin Tang and Robert Keller, 2009. [Learning Jazz Grammars](http://ai.stanford.edu/~kdtang/papers/smc09-jazzgrammar.pdf)
- Robert Keller and David Morrison, 2007, [A Grammatical Approach to Automatic Improvisation](http://smc07.uoa.gr/SMC07%20Proceedings/SMC07%20Paper%2055.pdf)
- François Pachet, 1999, [Surprising Harmonies](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.5.7473&rep=rep1&type=pdf)

We're also grateful to François Germain for valuable feedback.